In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from scipy.linalg import qr
from scipy.fftpack import idct, dct
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler
import seaborn as sns

plt.rcParams.update({'font.size': 14}) 


In [ ]:
file_path = ""  # Update with your actual file path
df = pd.read_csv(file_path, parse_dates=["datetime"])

df["datetime"] = pd.to_datetime(df["datetime"])

In [ ]:
def optimize_sensors(df, field, variable, scaler, figure, yticks, subfig):
    df = df[(df["field"] == field) &
        (df["datetime"].dt.year.isin([2023, 2024])) 
        &
        (df["datetime"].dt.month.isin([3, 4]))
        ]
    
    df = df[["datetime", "location", variable]]

    pivot_air = df.pivot(index="datetime", columns="location", values=variable)

    pivot_air = pivot_air.dropna()
    print(pivot_air)
    # Convert to matrix with rows = sensor nodes, columns = time
    WSN_matrix = pivot_air.to_numpy().T
    scaler = StandardScaler()

    WSN_matrix_scaled = scaler.fit_transform(WSN_matrix)

    num_sensors = 6
    num_timestamps = WSN_matrix_scaled.shape[1]  # Length of time-series

    # Assuming WSN_matrix_scaled is preprocessed & standardized

    # Extract timestamps from pivot_air
    timestamps = pivot_air.index  # Assuming pivot_air has datetime index

    # -----------------------------------------
    # QR Pivoting for Sensor Selection
    # -----------------------------------------
    def qr_pivoting(matrix, num_selected_sensors):
        """ Perform QR Pivoting to select optimal sensors. """
        _, _, pivoting = qr(matrix.T, pivoting=True)
        return pivoting[:num_selected_sensors]

    # -----------------------------------------
    # Construct Measurement Matrix (DCT Basis)
    # -----------------------------------------
    def construct_dct_matrix(sensor_indices, num_sensors):
        """ Construct a Discrete Cosine Transform (DCT) measurement matrix. """
        I = np.eye(num_sensors)
        Theta = np.array([dct(I[:, i], norm="ortho") for i in sensor_indices])
        return Theta

    # --------------------------------------
    # Sparse Recovery Problem Using L1-Minimization
    # ----------------------------------------
    def compressed_sensing_reconstruction(Theta, measurements):
        """ Solve L2-minimization problem using Lasso to recover missing sensor data. """
        a = cp.Variable(num_sensors)
        objective = cp.Minimize(cp.norm(a, 2))
        constraints = [Theta @ a == measurements]
        prob = cp.Problem(objective, constraints)
        prob.solve()
        return a.value  

    # -----------------------------------------
    # Loop Over Increasing Number of Selected Sensors
    # -----------------------------------------
    rmse_values = []
    sensor_counts = list(range(1, num_sensors))  # Varying number of sensors
    print(qr_pivoting(WSN_matrix_scaled, 6))

    for num_selected_sensors in sensor_counts:
        # Select training sensors using QR pivoting
        selected_sensors = qr_pivoting(WSN_matrix_scaled, num_selected_sensors)
        
        rmse_fold = []  # Store RMSE for each leave-one-out validation
        
        for validation_sensor in range(num_sensors):  # Leave-One-Out
            if validation_sensor in selected_sensors:
                continue  # Skip if already in selected sensors

            print(f"Training Sensors: {selected_sensors}, Validation Sensor: {validation_sensor}")
            
            # Training Data (Remove Validation Sensor)
            X_train = WSN_matrix_scaled[selected_sensors, :]
            X_valid = WSN_matrix_scaled[validation_sensor, :]  # Left-out sensor

            # Construct DCT Measurement Matrix
            Theta = construct_dct_matrix(selected_sensors, num_sensors)

            # Initialize Reconstructed Signal
            reconstructed_signal = np.zeros(num_timestamps)

            # Apply compressed sensing for each time step
            for t in range(num_timestamps):
                y_t = X_train[:, t]  # Observed measurements at time t
                a_opt = compressed_sensing_reconstruction(Theta, y_t)

                if a_opt is not None and len(a_opt) == num_sensors:
                    reconstructed_signal[t] = idct(a_opt, norm="ortho")[validation_sensor]
                else:
                    reconstructed_signal[t] = np.nan  # If reconstruction fails, assign NaN

            # Compute RMSE for this validation sensor
            true_signal = scaler.inverse_transform(WSN_matrix_scaled)[validation_sensor, :]
            reconstructed_signal_scaled = scaler.inverse_transform(reconstructed_signal.reshape(1, -1)).T

            rmse = np.sqrt(mean_squared_error(true_signal, reconstructed_signal_scaled))
            rmse_fold.append(rmse)

            print(f"Validation Sensor {validation_sensor}: RMSE = {rmse:.2f}")

        # Store the mean RMSE for this sensor count
        mean_rmse = np.mean(rmse_fold)
        rmse_values.append(mean_rmse)

        print(f"Sensors Used: {num_selected_sensors}, Mean RMSE: {mean_rmse:.2f}")

    # -----------------------------------------
    # Plot RMSE vs Number of Selected Sensors
    # -----------------------------------------


    # Find the first index where RMSE starts increasing
    increase_idx = np.where(np.diff(rmse_values) > 0)[0]  # Get increasing indices

    if len(increase_idx) > 0:
        first_increase_idx = increase_idx[0]  # Adjust for diff() shift
        highlight_x = sensor_counts[first_increase_idx]
        highlight_y = rmse_values[first_increase_idx]
    else:
        highlight_x, highlight_y = sensor_counts[-1], rmse_values[-1]  # No increasing RMSE found

    fig = plt.figure(figsize=(3, 3))
    plt.plot(sensor_counts, rmse_values, marker="o", linestyle="--", color="black")
    # Highlight the first increasing point
    if highlight_x is not None:
        plt.scatter(highlight_x, highlight_y, color="white", edgecolors="black", s=100, label="First Increase")

    plt.xlabel("Sensing Nodes")
    plt.ylabel("RMSE (°C)")
    plt.xticks(sensor_counts) 
    plt.yticks(yticks)
    plt.text(0.5, -0.25, subfig, ha='center', va='center', transform=plt.gca().transAxes, fontsize=8)
    fig.savefig(figure + '.jpg', dpi=600, bbox_inches='tight')
    plt.show()


In [ ]:
optimize_sensors(df, "benton-city", "air_temperature", figure='Figures/Figure 4a', yticks=np.arange(0.22, 0.3, 0.01), subfig="(a)")

In [ ]:
optimize_sensors(df, "benton-city", "leaf_temperature", figure='Figures/Figure 4b', yticks=np.arange(0.46, 0.52, 0.01), subfig='(b)')

In [ ]:
optimize_sensors(df, "sunnyside", "air_temperature", figure='Figures/Figure 4c', yticks=np.arange(1, 8, 1), subfig='(c)')

In [ ]:
optimize_sensors(df, "sunnyside", "leaf_temperature", figure='Figures/Figure 4d', yticks=np.arange(1, 8, 1), subfig='(d)')

In [ ]:
'''Sunnyside air temperature map '''

import geopandas as gpd
import pandas as pd
import seaborn as sns
from mpl_toolkits.basemap import Basemap
import matplotlib.patches as patches



# -------------------------------------------
# Load Sensor Metadata (Latitude, Longitude, Elevation)
# -------------------------------------------
# Assume you have a dataframe `sensor_metadata`
# Columns: ["sensor_id", "latitude", "longitude", "elevation"]
sensor_metadata = pd.read_csv(" ")  # Load sensor metadata

sensor_metadata.columns = ["id", "latitude", "longitude", "name", "site", "elevation"]

# -------------------------------------------
# Filter Sensors by Site (Only Sunnyside)
# -------------------------------------------
sunnyside_sensors = sensor_metadata[sensor_metadata["site"] == "sunnyside"]

# -------------------------------------------
# Select Sensors by ID (From QR-POD Optimization)
# -------------------------------------------
selected_sensor_ids = [3, 0, 2, 4]  # Example selected sensors

# Extract only the selected sensors from Sunnyside
selected_sensors_df = sunnyside_sensors[sunnyside_sensors["id"].isin(selected_sensor_ids)]

# -------------------------------------------
# Plot Selected Sensor Locations
# -------------------------------------------
fig = plt.figure(figsize=(5, 3))

# Initialize Basemap for Sunnyside region
m = Basemap(projection="merc",
            llcrnrlat=min(sunnyside_sensors["latitude"]) - 0.001,
            urcrnrlat=max(sunnyside_sensors["latitude"]) + 0.001,
            llcrnrlon=min(sunnyside_sensors["longitude"]) - 0.001,
            urcrnrlon=max(sunnyside_sensors["longitude"]) + 0.001,
            resolution="i")

m.drawcoastlines()
m.drawcountries()
m.drawrivers()
m.drawmapboundary(fill_color="lightblue")
m.fillcontinents(color="whitesmoke", lake_color="lightblue")

# Convert latitude/longitude to map coordinates
x, y = m(selected_sensors_df["longitude"].values, selected_sensors_df["latitude"].values)

# Plot Selected Sensors
plt.scatter(x, y, s=10)

elevations = selected_sensors_df["elevation"].values
sc = plt.scatter(x, y, s=100, c=elevations, cmap="terrain",marker="o", edgecolor="black", label="Nodes")

# Add colorbar
# cbar = plt.colorbar(sc)
# cbar.set_label("Elevation (m)")

ax = plt.gca()

wind_machine_df = sunnyside_sensors[sunnyside_sensors["name"].str.contains("wind-machine", case=False)]
x_wind, y_wind = m(wind_machine_df["longitude"].values, wind_machine_df["latitude"].values)
plt.scatter(x_wind, y_wind, s=150, c="black", marker="X", edgecolor="black", linewidth=1, label="Wind Machine")


plt.annotate("N", xy=(0.95, 0.84), xycoords="axes fraction", fontsize=14, fontweight="bold", ha="center", va="center")
# Define arrow location (top-right of the plot)
arrow_x = 0.95
arrow_y = 0.90

# Add a QGIS-style north arrow (bold triangle)
north_arrow = patches.RegularPolygon((arrow_x, arrow_y), numVertices=3, radius=0.03, orientation=not(np.pi),
                                     transform=ax.transAxes, color="black")
ax.add_patch(north_arrow)

scale_length_m = 50  # Define scale bar length in meters

# 🔹 Position for Scale Bar (bottom-right corner)
scale_lat_start = min(sunnyside_sensors["latitude"]) - 0.0005
scale_lon_start = max(sunnyside_sensors["longitude"]) - 0.00000001

# 🔹 Convert to Map Coordinates
scale_x_start, scale_y_start = m(scale_lon_start, scale_lat_start)
scale_x_end, _ = m(scale_lon_start + 0.0001333 * 3, scale_lat_start)  # Convert 10m to degrees

# 🔹 Draw the Scale Bar using Rectangle Patch
scale_bar_width = scale_x_end - scale_x_start
scale_bar_height = 4  # Adjust height for visibility

scale_bar = patches.Rectangle((scale_x_start, scale_y_start), scale_bar_width, scale_bar_height,
                              edgecolor="black", facecolor="black", linewidth=1)

# 🔹 Add Scale Bar to the Plot
ax = plt.gca()
ax.add_patch(scale_bar)

# 🔹 Add Scale Label (Centered on Scale Bar)
plt.text((scale_x_start + scale_x_end) / 2, scale_y_start - 20, f"{scale_length_m} m",
         fontsize=10, ha="center", va="top", color="black")

# plt.subplots_adjust(bottom=-0.2)  # Increase the bottom margin (adjust as needed)

text = plt.text(0.5, -0.1, "(c)", ha='center', va='center', transform=plt.gca().transAxes, fontsize=8)
# Titles & Legend
fig.savefig("Figures/Figure 5c" + '.jpg', dpi=600, bbox_inches='tight', bbox_extra_artists=[text])

plt.show()

In [ ]:
'''Sunnyside bud temperature map '''

import geopandas as gpd
import pandas as pd
import seaborn as sns
from mpl_toolkits.basemap import Basemap
import matplotlib.patches as patches



# -------------------------------------------
# Load Sensor Metadata (Latitude, Longitude, Elevation)
# -------------------------------------------
# Assume you have a dataframe `sensor_metadata`
# Columns: ["sensor_id", "latitude", "longitude", "elevation"]
sensor_metadata = pd.read_csv(" ")  # Load sensor metadata

sensor_metadata.columns = ["id", "latitude", "longitude", "name", "site", "elevation"]

# -------------------------------------------
# Filter Sensors by Site (Only Sunnyside)
# -------------------------------------------
sunnyside_sensors = sensor_metadata[sensor_metadata["site"] == "sunnyside"]

# -------------------------------------------
# Select Sensors by ID (From QR-POD Optimization)
# -------------------------------------------
selected_sensor_ids = [3, 0, 2, 4, 5, 1]  # Example selected sensors

# Extract only the selected sensors from Sunnyside
selected_sensors_df = sunnyside_sensors[sunnyside_sensors["id"].isin(selected_sensor_ids)]

# -------------------------------------------
# Plot Selected Sensor Locations
# -------------------------------------------
fig = plt.figure(figsize=(5, 3))

# Initialize Basemap for Sunnyside region
m = Basemap(projection="merc",
            llcrnrlat=min(sunnyside_sensors["latitude"]) - 0.001,
            urcrnrlat=max(sunnyside_sensors["latitude"]) + 0.001,
            llcrnrlon=min(sunnyside_sensors["longitude"]) - 0.001,
            urcrnrlon=max(sunnyside_sensors["longitude"]) + 0.001,
            resolution="i")

m.drawcoastlines()
m.drawcountries()
m.drawrivers()
m.drawmapboundary(fill_color="lightblue")
m.fillcontinents(color="whitesmoke", lake_color="lightblue")

# Convert latitude/longitude to map coordinates
x, y = m(selected_sensors_df["longitude"].values, selected_sensors_df["latitude"].values)

# Plot Selected Sensors
plt.scatter(x, y, s=10)

elevations = selected_sensors_df["elevation"].values
sc = plt.scatter(x, y, s=100, c=elevations, cmap="terrain",marker="o", edgecolor="black", label="Nodes")

# Add colorbar
cbar = plt.colorbar(sc)
cbar.set_label("Elevation (m)")

ax = plt.gca()

wind_machine_df = sunnyside_sensors[sunnyside_sensors["name"].str.contains("wind-machine", case=False)]
x_wind, y_wind = m(wind_machine_df["longitude"].values, wind_machine_df["latitude"].values)
plt.scatter(x_wind, y_wind, s=150, c="black", marker="X", edgecolor="black", linewidth=1, label="Wind Machine")


plt.annotate("N", xy=(0.95, 0.84), xycoords="axes fraction", fontsize=14, fontweight="bold", ha="center", va="center")
# Define arrow location (top-right of the plot)
arrow_x = 0.95
arrow_y = 0.90

# Add a QGIS-style north arrow (bold triangle)
north_arrow = patches.RegularPolygon((arrow_x, arrow_y), numVertices=3, radius=0.03, orientation=not(np.pi),
                                     transform=ax.transAxes, color="black")
ax.add_patch(north_arrow)

scale_length_m = 50  # Define scale bar length in meters

# 🔹 Position for Scale Bar (bottom-right corner)
scale_lat_start = min(sunnyside_sensors["latitude"]) - 0.0005
scale_lon_start = max(sunnyside_sensors["longitude"]) - 0.00000001

# 🔹 Convert to Map Coordinates
scale_x_start, scale_y_start = m(scale_lon_start, scale_lat_start)
scale_x_end, _ = m(scale_lon_start + 0.0001333 * 3, scale_lat_start)  # Convert 10m to degrees

# 🔹 Draw the Scale Bar using Rectangle Patch
scale_bar_width = scale_x_end - scale_x_start
scale_bar_height = 4  # Adjust height for visibility

scale_bar = patches.Rectangle((scale_x_start, scale_y_start), scale_bar_width, scale_bar_height,
                              edgecolor="black", facecolor="black", linewidth=1)

# 🔹 Add Scale Bar to the Plot
ax = plt.gca()
ax.add_patch(scale_bar)

# 🔹 Add Scale Label (Centered on Scale Bar)
plt.text((scale_x_start + scale_x_end) / 2, scale_y_start - 20, f"{scale_length_m} m",
         fontsize=10, ha="center", va="top", color="black")

plt.text(0.5, -0.1, "(d)", ha='center', va='center', transform=plt.gca().transAxes, fontsize=8)

# Titles & Legend
fig.savefig("Figures/Figure 5d" + '.jpg', dpi=600, bbox_inches='tight')

plt.show()

In [ ]:
'''Benton City Bud temperautre optimized nodes'''

sensor_metadata = pd.read_csv(" ")  # Load sensor metadata

sensor_metadata.columns = ["id", "latitude", "longitude", "name", "site", "elevation"]

# -------------------------------------------
# Filter Sensors by Site (Only Sunnyside)
# -------------------------------------------
sunnyside_sensors = sensor_metadata[sensor_metadata["site"] == "benton-city"]

# -------------------------------------------
#  Select Sensors by ID (From QR-POD Optimization)
# -------------------------------------------
selected_sensor_ids = [3, 0, 4]  # Example selected sensors

# Extract only the selected sensors from Sunnyside
selected_sensors_df = sunnyside_sensors[sunnyside_sensors["id"].isin(selected_sensor_ids)]

# -------------------------------------------
#  Plot Selected Sensor Locations
# -------------------------------------------
fig = plt.figure(figsize=(5, 3))

# Initialize Basemap for Sunnyside region
m = Basemap(projection="merc",
            llcrnrlat=min(sunnyside_sensors["latitude"]) - 0.001,
            urcrnrlat=max(sunnyside_sensors["latitude"]) + 0.001,
            llcrnrlon=min(sunnyside_sensors["longitude"]) - 0.001,
            urcrnrlon=max(sunnyside_sensors["longitude"]) + 0.001,
            resolution="i")

m.drawcoastlines()
m.drawcountries()
m.drawrivers()
m.drawmapboundary(fill_color="lightblue")
m.fillcontinents(color="whitesmoke", lake_color="lightblue")

# Convert latitude/longitude to map coordinates
x, y = m(selected_sensors_df["longitude"].values, selected_sensors_df["latitude"].values)

# Plot Selected Sensors
plt.scatter(x, y, s=10)

elevations = selected_sensors_df["elevation"].values
sc = plt.scatter(x, y, s=100, c=elevations, cmap="terrain", edgecolor="black", label="Nodes")

# Add colorbar
cbar = plt.colorbar(sc)
cbar.set_label("Elevation (m)")

ax = plt.gca()

wind_machine_df = sunnyside_sensors[sunnyside_sensors["name"].str.contains("wind-machine", case=False)]
x_wind, y_wind = m(wind_machine_df["longitude"].values, wind_machine_df["latitude"].values)
plt.scatter(x_wind, y_wind, s=150, c="black", marker="X", edgecolor="black", linewidth=1, label="Wind Machine")


plt.annotate("N", xy=(0.95, 0.84), xycoords="axes fraction", fontsize=14, fontweight="bold", ha="center", va="center")
# Define arrow location (top-right of the plot)
arrow_x = 0.95
arrow_y = 0.90

# Add a QGIS-style north arrow (bold triangle)
north_arrow = patches.RegularPolygon((arrow_x, arrow_y), numVertices=3, radius=0.03, orientation=not(np.pi),
                                     transform=ax.transAxes, color="black")
ax.add_patch(north_arrow)


scale_length_m = 50  # Define scale bar length in meters

# 🔹 Position for Scale Bar (bottom-right corner)
scale_lat_start = min(sunnyside_sensors["latitude"]) - 0.0005
scale_lon_start = max(sunnyside_sensors["longitude"]) - 0.00000001

# 🔹 Convert to Map Coordinates
scale_x_start, scale_y_start = m(scale_lon_start, scale_lat_start)
scale_x_end, _ = m(scale_lon_start + 0.0001333 * 5, scale_lat_start)  # Convert 10m to degrees

# 🔹 Draw the Scale Bar using Rectangle Patch
scale_bar_width = scale_x_end - scale_x_start
scale_bar_height = 4  # Adjust height for visibility

scale_bar = patches.Rectangle((scale_x_start, scale_y_start), scale_bar_width, scale_bar_height,
                              edgecolor="black", facecolor="black", linewidth=1)

# 🔹 Add Scale Bar to the Plot
ax = plt.gca()
ax.add_patch(scale_bar)

# 🔹 Add Scale Label (Centered on Scale Bar)
plt.text((scale_x_start + scale_x_end) / 2, scale_y_start - 20, f"{scale_length_m} m",
         fontsize=10, ha="center", va="top", color="black")


plt.text(0.5, -0.1, "(b)", ha='center', va='center', transform=plt.gca().transAxes, fontsize=8)

# Titles & Legend
fig.savefig("Figures/Figure 5b" + '.jpg', dpi=600, bbox_inches='tight')

plt.show()

In [ ]:
'''Benton City Air temperautre optimized nodes'''


sensor_metadata = pd.read_csv(" ")  # Load sensor metadata

sensor_metadata.columns = ["id", "latitude", "longitude", "name", "site", "elevation"]

# -------------------------------------------
# Filter Sensors by Site (Only Sunnyside)
# -------------------------------------------
sunnyside_sensors = sensor_metadata[sensor_metadata["site"] == "benton-city"]

# -------------------------------------------
# Select Sensors by ID (From QR-POD Optimization)
# -------------------------------------------
selected_sensor_ids = [0, 4]  # Example selected sensors

# Extract only the selected sensors from Sunnyside
selected_sensors_df = sunnyside_sensors[sunnyside_sensors["id"].isin(selected_sensor_ids)]

# -------------------------------------------
# Plot Selected Sensor Locations
# -------------------------------------------
fig = plt.figure(figsize=(5, 3))

# Initialize Basemap for Sunnyside region
m = Basemap(projection="merc",
            llcrnrlat=min(sunnyside_sensors["latitude"]) - 0.001,
            urcrnrlat=max(sunnyside_sensors["latitude"]) + 0.001,
            llcrnrlon=min(sunnyside_sensors["longitude"]) - 0.001,
            urcrnrlon=max(sunnyside_sensors["longitude"]) + 0.001,
            resolution="i")

m.drawcoastlines()
m.drawcountries()
m.drawrivers()
m.drawmapboundary(fill_color="lightblue")
m.fillcontinents(color="whitesmoke", lake_color="lightblue")

# Convert latitude/longitude to map coordinates
x, y = m(selected_sensors_df["longitude"].values, selected_sensors_df["latitude"].values)

# Plot Selected Sensors
plt.scatter(x, y, s=10)

elevations = selected_sensors_df["elevation"].values
sc = plt.scatter(x, y, s=100, c=elevations, cmap="terrain", edgecolor="black", label="SN")

# Add colorbar
# cbar = plt.colorbar()
# cbar.set_label("Elevation (m)")

ax = plt.gca()

wind_machine_df = sunnyside_sensors[sunnyside_sensors["name"].str.contains("wind-machine", case=False)]
x_wind, y_wind = m(wind_machine_df["longitude"].values, wind_machine_df["latitude"].values)
plt.scatter(x_wind, y_wind, s=150, c="black", marker="X", edgecolor="black", linewidth=1, label="WM")


plt.annotate("N", xy=(0.95, 0.84), xycoords="axes fraction", fontsize=14, fontweight="bold", ha="center", va="center")
# Define arrow location (top-right of the plot)
arrow_x = 0.95
arrow_y = 0.90

# Add a QGIS-style north arrow (bold triangle)
north_arrow = patches.RegularPolygon((arrow_x, arrow_y), numVertices=3, radius=0.03, orientation=not(np.pi),
                                     transform=ax.transAxes, color="black")
ax.add_patch(north_arrow)


scale_length_m = 50  # Define scale bar length in meters

# Position for Scale Bar (bottom-right corner)
scale_lat_start = min(sunnyside_sensors["latitude"]) - 0.0005
scale_lon_start = max(sunnyside_sensors["longitude"]) - 0.00000001

# Convert to Map Coordinates
scale_x_start, scale_y_start = m(scale_lon_start, scale_lat_start)
scale_x_end, _ = m(scale_lon_start + 0.0001333 * 5, scale_lat_start)  # Convert 10m to degrees

# Draw the Scale Bar using Rectangle Patch
scale_bar_width = scale_x_end - scale_x_start
scale_bar_height = 4  # Adjust height for visibility

scale_bar = patches.Rectangle((scale_x_start, scale_y_start), scale_bar_width, scale_bar_height,
                              edgecolor="black", facecolor="black", linewidth=1)

# Add Scale Bar to the Plot
ax = plt.gca()
ax.add_patch(scale_bar)

# Add Scale Label (Centered on Scale Bar)
plt.text((scale_x_start + scale_x_end) / 2, scale_y_start - 20, f"{scale_length_m} m",
         fontsize=10, ha="center", va="top", color="black")

# Titles & Legend
plt.legend()
text = plt.text(0.5, -0.1, "(a)", ha='center', va='center', transform=plt.gca().transAxes, fontsize=8)

fig.savefig("Figures/Figure 5a" + '.jpg', dpi=600, bbox_inches='tight', bbox_extra_artists=[text])

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft, fftfreq

file_path = " "  # Update with your actual file path
df = pd.read_csv(file_path, parse_dates=["datetime"])

df["datetime"] = pd.to_datetime(df["datetime"])

field = 'sunnyside'
df = df[(df["field"] == field) &
        (df["datetime"].dt.year.isin([2024])) 
        &
        (df["datetime"].dt.month.isin([3, 4]))
        ]

df = df[["datetime", "location", "air_temperature", "leaf_temperature"]]
df = df[(df['datetime'].dt.hour >= 0) & (df['datetime'].dt.hour < 8)]

pivot_air = df.pivot(index="datetime", columns="location", values="air_temperature")
pivot_leaf = df.pivot(index="datetime", columns="location", values="leaf_temperature")
pivot_air.interpolate(method='linear', inplace=True, axis=1)
pivot_leaf.interpolate(method='linear', inplace=True, axis=1)

pivot_air = pivot_air.dropna()
pivot_leaf = pivot_leaf.dropna()
# ---------------------------------------
# 1️⃣ Load Your Temperature Data
# ---------------------------------------
# Assuming pivot_air and pivot_leaf are (timestamps x nodes) DataFrames
# where each column represents a sensor node and each row represents a time step.

# Convert data to numpy arrays
air_temp_data = pivot_air.values  # Shape: (time, nodes)
leaf_temp_data = pivot_leaf.values  # Shape: (time, nodes)

# Sampling frequency (assume hourly data)
dt = 5/60  # Time step in hours
Fs = 1 / dt  # Sampling frequency (per hour)
N = air_temp_data.shape[0]  # Number of time points
freqs = fftfreq(N, d=dt)  # Frequency bins

# Compute Fourier transforms
fft_air = fft(air_temp_data, axis=0)
fft_leaf = fft(leaf_temp_data, axis=0)

# Compute magnitudes and phases
magnitude_air = np.abs(fft_air)
magnitude_leaf = np.abs(fft_leaf)
phase_air = np.angle(fft_air)
phase_leaf = np.angle(fft_leaf)

# Compute phase difference at diurnal frequency (1 cycle per 24 hours)
diurnal_freq_idx = np.argmin(np.abs(freqs - 1/0.5))  # Closest to 1/24 hour^-1
wind_machine_index = 5  

# Compute phase difference relative to the wind machine
phase_diff_air_diurnal = phase_air[diurnal_freq_idx, :] - phase_air[diurnal_freq_idx, wind_machine_index]
phase_diff_leaf_diurnal = phase_leaf[diurnal_freq_idx, :] - phase_leaf[diurnal_freq_idx, wind_machine_index]

# Convert phase difference to time lag in hours
time_lag_air_hours = phase_diff_air_diurnal / (2 * np.pi * (1))  # Convert to hours
time_lag_leaf_hours = phase_diff_leaf_diurnal / (2 * np.pi * (1))  # Convert to hours

num_sensors = air_temp_data.shape[1]

x = np.arange(num_sensors)  # Sensor indices
width = 0.3  # Width of bars

# ---------------------------------------
# Plot Magnitude Spectrum (High & Low Frequencies)
# ---------------------------------------

low_freq_range = (freqs > 0) & (freqs < 1/6)  # Below 6-hour periodicity
high_freq_range = (freqs > 1/6) & (freqs < 1/0.5)  # Between 6-hour and 2-hour periodicity


fig = plt.figure(figsize=(3, 2))
# High-Frequency Components (Short-Term Variability)
plt.bar(x - width/2, np.mean(magnitude_air[high_freq_range, :], axis=0), width=width, alpha=0.9, color="darkorange", label="Air Temperature")
plt.bar(x + width/2, np.mean(magnitude_leaf[high_freq_range, :], axis=0), width=width, alpha=0.9, color="darkgreen", label="Bud Tissue Temperature")
plt.xlabel("Sensor Node")
plt.ylabel("Mean Magnitude")
# plt.title("High Frequency Components")
plt.xticks(x, ['SN1', 'SN2', 'SN3', 'SN4', 'SN5', 'SN6'])
plt.yticks(range(0, 150, 20))

# plt.legend()
plt.text(0.5, -0.4, '(b)', ha='center', va='center', transform=plt.gca().transAxes, fontsize=8)

fig.savefig("Figures/Figure 2b" + '.jpg', dpi=600, bbox_inches='tight')
# ---------------------------------------
# Plot Phase Difference at Diurnal Frequency
# ---------------------------------------

x = np.arange(5)  # Sensor indices
width = 0.2
fig = plt.figure(figsize=(3, 2))

# Plot air temperature phase lag (left bars)
plt.bar(x - width/2, -np.abs(time_lag_air_hours[:-1]), width=width, label="Air Temperature", color="darkorange")

# Plot leaf temperature phase lag (right bars)
plt.bar(x + width/2,  -np.abs(time_lag_leaf_hours[:-1]), width=width, label="Bud Tissue Temperature", color="darkgreen")

plt.xlabel("Sensor Node")
plt.ylabel("Time Lag (H)")
# plt.title("Phase Lag at Diurnal Frequency (Relative to Wind Machine)")

plt.xticks(x, ['SN1', 'SN2', 'SN3', 'SN4', 'SN5'])
# plt.legend()
# plt.grid(True, linestyle=':', alpha=0.6)
plt.text(0.5, -0.4, '(d)', ha='center', va='center', transform=plt.gca().transAxes, fontsize=8)
fig.savefig("Figures/Figure 2d" + '.jpg', dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft, fftfreq

file_path = " "  # Update with your actual file path
df = pd.read_csv(file_path, parse_dates=["datetime"])

df["datetime"] = pd.to_datetime(df["datetime"])

field = 'benton-city'
df = df[(df["field"] == field) &
        (df["datetime"].dt.year.isin([2023, 2024])) 
        &
        (df["datetime"].dt.month.isin([3, 4]))
        ]

df = df[["datetime", "location", "air_temperature", "leaf_temperature"]]
df = df[(df['datetime'].dt.hour >= 0) & (df['datetime'].dt.hour < 8)]

pivot_air = df.pivot(index="datetime", columns="location", values="air_temperature")
pivot_leaf = df.pivot(index="datetime", columns="location", values="leaf_temperature")
pivot_air.interpolate(method='linear', inplace=True, axis=1)
pivot_leaf.interpolate(method='linear', inplace=True, axis=1)

pivot_air = pivot_air.dropna()
pivot_leaf = pivot_leaf.dropna()
# ---------------------------------------
# Load Your Temperature Data
# ---------------------------------------
# Assuming pivot_air and pivot_leaf are (timestamps x nodes) DataFrames
# where each column represents a sensor node and each row represents a time step.

# Convert data to numpy arrays
air_temp_data = pivot_air.values  # Shape: (time, nodes)
leaf_temp_data = pivot_leaf.values  # Shape: (time, nodes)

# Sampling frequency (assume hourly data)
dt = 5/60  # Time step in hours
Fs = 1 / dt  # Sampling frequency (per hour)
N = air_temp_data.shape[0]  # Number of time points
freqs = fftfreq(N, d=dt)  # Frequency bins

# Compute Fourier transforms
fft_air = fft(air_temp_data, axis=0)
fft_leaf = fft(leaf_temp_data, axis=0)

# Compute magnitudes and phases
magnitude_air = np.abs(fft_air)
magnitude_leaf = np.abs(fft_leaf)
phase_air = np.angle(fft_air)
phase_leaf = np.angle(fft_leaf)

# Compute phase difference at diurnal frequency (1 cycle per 24 hours)
diurnal_freq_idx = np.argmin(np.abs(freqs - 1/2))  # Closest to 1/24 hour^-1
wind_machine_index = 5  

# Compute phase difference relative to the wind machine
phase_diff_air_diurnal = phase_air[diurnal_freq_idx, :] - phase_air[diurnal_freq_idx, wind_machine_index]
phase_diff_leaf_diurnal = phase_leaf[diurnal_freq_idx, :] - phase_leaf[diurnal_freq_idx, wind_machine_index]

# Convert phase difference to time lag in hours
time_lag_air_hours = phase_diff_air_diurnal / (2 * np.pi * (1/2))  # Convert to hours
time_lag_leaf_hours = phase_diff_leaf_diurnal / (2 * np.pi * (1/2))  # Convert to hours

num_sensors = air_temp_data.shape[1]

x = np.arange(num_sensors)  # Sensor indices
width = 0.3  # Width of bars

# ---------------------------------------
# Plot Magnitude Spectrum (High & Low Frequencies)
# ---------------------------------------

low_freq_range = (freqs > 0) & (freqs < 1/6)  # Below 6-hour periodicity
high_freq_range = (freqs > 1/6) & (freqs < 1/0.5)  # Between 6-hour and 2-hour periodicity


fig = plt.figure(figsize=(3, 2))
# High-Frequency Components (Short-Term Variability)
plt.bar(x - width/2, np.mean(magnitude_air[high_freq_range, :], axis=0), width=width, alpha=0.9, color="darkorange", label=r'$T_{a}$')
plt.bar(x + width/2, np.mean(magnitude_leaf[high_freq_range, :], axis=0), width=width, alpha=0.9, color="darkgreen", label=r'$T_{b}$')
plt.xlabel("Sensor Node")
plt.ylabel("Mean Magnitude")
# plt.title("High Frequency Components")
plt.xticks(x, ['SN1', 'SN2', 'SN3', 'SN4', 'SN5', 'SN6'])
plt.yticks(range(0, 150, 20))
plt.legend(loc='upper left')
# Add subcaption (a) at the bottom center
plt.text(0.5, -0.4, '(a)', ha='center', va='center', transform=plt.gca().transAxes, fontsize=8)
# plt.grid(True, linestyle=":", alpha=0.6)
fig.savefig("Figures/Figure 2a" + '.jpg', dpi=600, bbox_inches='tight')
# ---------------------------------------
# Plot Phase Difference at Diurnal Frequency
# ---------------------------------------

x = np.arange(5)  # Sensor indices
width = 0.2
fig = plt.figure(figsize=(3, 2))

# Plot air temperature phase lag (left bars)
plt.bar(x - width/2, -np.abs(time_lag_air_hours[:-1]), width=width, label=r'$T_{a}$', color="darkorange")

# Plot leaf temperature phase lag (right bars)
plt.bar(x + width/2,  -np.abs(time_lag_leaf_hours[:-1]), width=width, label=r'$T_{b}$', color="darkgreen")

plt.xlabel("Sensor Node")
plt.ylabel("Time Lag (H)")
# plt.title("Phase Lag at Diurnal Frequency (Relative to Wind Machine)")

plt.xticks(x, ['SN1', 'SN2', 'SN3', 'SN4', 'SN5'])  # Ensure correct x-axis labels

# plt.legend()
# plt.grid(True, linestyle=':', alpha=0.6)
# plt.tight_layout()
plt.text(0.5, -0.4, '(c)', ha='center', va='center', transform=plt.gca().transAxes, fontsize=8)

fig.savefig("Figures/Figure 2c" + '.jpg', dpi=600, bbox_inches='tight')
plt.show()



In [ ]:
weather_file = ' '
ch_model_file = ' '

# Read weather data (5-minute intervals)
weather_df = pd.read_csv(weather_file, parse_dates=['datetime'])

weather_df['datetime'] = pd.to_datetime(weather_df['datetime'], errors='coerce')

weather_df = weather_df[(weather_df['datetime'].dt.year > 2022)]

weather_df = weather_df[(weather_df['datetime'].dt.hour > 0) &  (weather_df['datetime'].dt.hour < 8)]
# Extract Date from datetime for joining
weather_df['Date'] = weather_df['datetime'].dt.normalize()  

# Read combined CH model data (with LT10 thresholds)
ch_model_df = pd.read_csv(ch_model_file, parse_dates=['Date'])

ch_model_df['Date'] = pd.to_datetime(ch_model_df['Date'], errors='coerce') 

# Merge on 'Date' and 'field'
merged_df = pd.merge(weather_df, ch_model_df, on=['Date', 'field'], how='inner')

def compute_csdi(df):
    """
    Computes the Cold Spell Duration Index (CSDI) per day.
    
    Parameters:
    - df: DataFrame with 'datetime', 'bud_temp', and 'field'.
    - threshold: Temperature threshold for cold spell (e.g., LT10 value).

    Returns:
    - DataFrame with CSDI per day and field.
    """
    # Flag cold events where bud temperature is below threshold
    df['cold_event'] = df['air_temperature'] < df['chelan_LT10'] + 0.5

    df['csmi'] = df.apply(lambda row: (row['chelan_LT10'] - row['air_temperature']) * (5 / 60) 
                          if row['air_temperature'] < row['chelan_LT10'] else 0, axis=1)


    # Group by date and field to sum durations
    csdi_df = df.groupby(['Date', 'field', 'location']).agg(
        cold_duration=('cold_event', 'sum'),  # Count cold events (each row is a 5-min interval)
        thrm_bud_min = ('thrm_bud_temp', 'min'),
        chelan_LT10 = ('chelan_LT10', 'min'),
        CSMI_sum=('csmi', 'sum'),  # Sum of CSMI over the day
        CSMI_mean=('csmi', 'mean'),  # Mean CSMI per day
        CSMI_max=('csmi', 'max'),  # Max CSMI per day
        CSMI_min=('csmi', 'min')   # Min CSMI per day
    ).reset_index()



    # Convert count to duration in hours (5 min intervals)
    csdi_df['cold_duration_hours'] = csdi_df['cold_duration'] * (5 / 60)

    # Normalize the CSDI (0 to 1 scale)
    max_duration = csdi_df['cold_duration_hours'].max()
    csdi_df['csdi_normalized'] = csdi_df['cold_duration_hours'] / max_duration if max_duration != 0 else 0

    # Normalize CSMI for each field
    def normalize_csmi(df):
        max_csmi = df['CSMI_max'].max()
        min_csmi = df['CSMI_min'].min()
        df['csmi_normalized'] = (
            (df['CSMI_max'] - df['CSMI_min']) / (df['CSMI_sum'] - df['CSMI_min'])
            if max_csmi != min_csmi else 0
        )
        return df
    
    csdi_df = csdi_df.groupby(['Date', 'field', 'location']).apply(normalize_csmi).reset_index(drop=True)

    return csdi_df


daily_csdi = compute_csdi(merged_df)

filtered_data = daily_csdi[daily_csdi['field'] == 'benton-city']
filtered_data = filtered_data[filtered_data['cold_duration'] > 0]

# Rename nodes to SN1, SN2, ..., SN6
node_mapping = {
    'node-1': 'SN1',
    'node-2': 'SN2',
    'node-3': 'SN3',
    'node-4': 'SN4',
    'node-5': 'SN5',
    'wind-machine': 'SN6'
}
filtered_data['location'] = filtered_data['location'].map(node_mapping)

pivot_data = filtered_data.pivot(index='Date', columns='location', values=['cold_duration_hours', 'thrm_bud_min'])


# Plotting
fig, ax1 = plt.subplots(figsize=(3, 2))

# Bar plot for cold_duration_hours
pivot_data['cold_duration_hours'].plot(kind='bar', ax=ax1, width=0.8, alpha=0.7, legend=False)

# Set x-ticks to only dates with data
ax1.set_xticks(range(len(pivot_data.index)))
ax1.set_xticklabels(pivot_data.index.strftime('%Y-%m-%d'), rotation=90)

# Add labels and title
ax1.set_xlabel('Date (Y-m-d)')
ax1.set_ylabel('ACH (Hours)')
# ax1.set_title('Cold Duration Hours and Minimum Temperatures by Date and Location')

ax1.set_yticks(range(7))

lines, labels = ax1.get_legend_handles_labels()

# Adjust layout
plt.text(0.5, -0.8, '(a)', ha='center', va='center', transform=plt.gca().transAxes, fontsize=8)

fig.savefig("Figures/Figure 3a" + '.jpg', dpi=600, bbox_inches='tight')

# Show the plot
plt.show()

In [ ]:
weather_file = ' '
ch_model_file = ' '

# Read weather data (5-minute intervals)
weather_df = pd.read_csv(weather_file, parse_dates=['datetime'])

weather_df['datetime'] = pd.to_datetime(weather_df['datetime'], errors='coerce')

weather_df = weather_df[(weather_df['datetime'].dt.year > 2022)]

weather_df = weather_df[(weather_df['datetime'].dt.hour > 0) &  (weather_df['datetime'].dt.hour < 8)]
# Extract Date from datetime for joining
weather_df['Date'] = weather_df['datetime'].dt.normalize()  

# Read combined CH model data (with LT10 thresholds)
ch_model_df = pd.read_csv(ch_model_file, parse_dates=['Date'])

ch_model_df['Date'] = pd.to_datetime(ch_model_df['Date'], errors='coerce') 

# Merge on 'Date' and 'field'
merged_df = pd.merge(weather_df, ch_model_df, on=['Date', 'field'], how='inner')

def compute_csdi(df):
    """
    Computes the Cold Spell Duration Index (CSDI) per day.
    
    Parameters:
    - df: DataFrame with 'datetime', 'bud_temp', and 'field'.
    - threshold: Temperature threshold for cold spell (e.g., LT10 value).

    Returns:
    - DataFrame with CSDI per day and field.
    """
    # Flag cold events where bud temperature is below threshold
    df['cold_event'] = df['air_temperature'] < df['chelan_LT10'] + 0.5

    df['csmi'] = df.apply(lambda row: (row['chelan_LT10'] - row['air_temperature']) * (5 / 60) 
                          if row['air_temperature'] < row['chelan_LT10'] else 0, axis=1)


    # Group by date and field to sum durations
    csdi_df = df.groupby(['Date', 'field', 'location']).agg(
        cold_duration=('cold_event', 'sum'),  # Count cold events (each row is a 5-min interval)
        thrm_bud_min = ('thrm_bud_temp', 'min'),
        chelan_LT10 = ('chelan_LT10', 'min'),
        CSMI_sum=('csmi', 'sum'),  # Sum of CSMI over the day
        CSMI_mean=('csmi', 'mean'),  # Mean CSMI per day
        CSMI_max=('csmi', 'max'),  # Max CSMI per day
        CSMI_min=('csmi', 'min')   # Min CSMI per day
    ).reset_index()



    # Convert count to duration in hours (5 min intervals)
    csdi_df['cold_duration_hours'] = csdi_df['cold_duration'] * (5 / 60)

    # Normalize the CSDI (0 to 1 scale)
    max_duration = csdi_df['cold_duration_hours'].max()
    csdi_df['csdi_normalized'] = csdi_df['cold_duration_hours'] / max_duration if max_duration != 0 else 0

    # Normalize CSMI for each field
    def normalize_csmi(df):
        max_csmi = df['CSMI_max'].max()
        min_csmi = df['CSMI_min'].min()
        df['csmi_normalized'] = (
            (df['CSMI_max'] - df['CSMI_min']) / (df['CSMI_sum'] - df['CSMI_min'])
            if max_csmi != min_csmi else 0
        )
        return df
    
    csdi_df = csdi_df.groupby(['Date', 'field', 'location']).apply(normalize_csmi).reset_index(drop=True)

    return csdi_df


daily_csdi = compute_csdi(merged_df)

filtered_data = daily_csdi[daily_csdi['field'] == 'sunnyside']
filtered_data = filtered_data[filtered_data['cold_duration'] > 0]

# Rename nodes to SN1, SN2, ..., SN6
node_mapping = {
    'node-1': 'SN1',
    'node-2': 'SN2',
    'node-3': 'SN3',
    'node-4': 'SN4',
    'node-5': 'SN5',
    'wind-machine': 'SN6'
}
filtered_data['location'] = filtered_data['location'].map(node_mapping)

filtered_data = filtered_data[filtered_data['Date'].dt.year > 2023]

pivot_data = filtered_data.pivot(index='Date', columns='location', values=['cold_duration_hours', 'thrm_bud_min'])


# Plotting
fig, ax1 = plt.subplots(figsize=(8, 4))

pivot_data['cold_duration_hours'].plot(kind='bar', ax=ax1, width=0.8, alpha=0.7)

idx = pivot_data.index  # DatetimeIndex

# -------- Lower ticks: the actual dates --------
ax1.set_xticks(range(len(idx)))
ax1.set_xticklabels(idx.strftime('%d'), rotation=45)

ax1.set_yticks(range(7))

# Add labels and title
ax1.set_xlabel('Date (d)',fontsize=14)
ax1.set_ylabel('ACH (Hours)', fontsize=14)
ax1.tick_params(axis='both', labelsize=14)
ax1.legend(fontsize=14)

# Adjust layout
# plt.text(0.5, -0.8, '(b)', ha='center', va='center', transform=plt.gca().transAxes, fontsize=8)

fig.savefig("ACD Plot Y24" + '.jpg', dpi=600, bbox_inches='tight')

# Show the plot
plt.show()

